## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from itertools import islice

import pickle as pkl

import sentencepiece as sp

import tkseem as tk
import gensim

/home/ellzo/.local/lib/python3.11/site-packages/matplotlib/projections/__init__.py:63: UserWarning: Unable to import Axes3D. This may be due to multiple versions of Matplotlib being installed (e.g. as a system package and as a pip package). As a result, the 3D projection is not available.
  warnings.warn("Unable to import Axes3D. This may be due to multiple versions of "


In [2]:
df = pd.read_csv('all_poems_clean.csv')
df['poem_text'] = df['poem_text'].astype(str)
df.head()

,poem_text
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...


## Morphological Tokenizer

In [3]:
morph_tokenizer = tk.MorphologicalTokenizer(vocab_size=1_000_000)

morph_tokenizer.train()

Training MorphologicalTokenizer ...


In [4]:
def tokenize_text_morph(text):
    return morph_tokenizer.tokenize(text)

df['morph_t'] = df.poem_text.apply(tokenize_text_morph)
df.head()

,poem_text,morph_t
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,"[عينا, ##ك, غابتا, نخيل, ساعة, ال, ##سحر, <UNK..."
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,"[<UNK>, لا, <UNK>, و, في, يدي, قدحي, ياليل, <U..."
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,"[علي, مقلتي, ##ك, ارتشفت, النج, ##و, ##م, وعا,..."
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,"[اساط, ##ير, من, حشرجات, ال, ##زمان, نسيج, ال,..."
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,"[والت, ##ف, حول, ##ك, ساعدا, ##ي, وم, ##ال, جي..."


In [5]:
morph_model = gensim.models.Word2Vec(
    df['morph_t'],
    window=5,
    min_count=4,
    workers=8)

In [6]:
morph_model.wv.most_similar(positive='ثلج')

[('غيوم', 0.8489223718643188),
 ('مياه', 0.828999400138855),
 ('عصفورة', 0.8215725421905518),
 ('عشب', 0.8178067803382874),
 ('فراشات', 0.8166794180870056),
 ('دفء', 0.8151868581771851),
 ('زجاج', 0.8145820498466492),
 ('مطر', 0.8118013143539429),
 ('رعشة', 0.8115999102592468),
 ('غيمة', 0.8035085797309875)]

In [7]:
morph_model.save('models/morph_word2vec_model.model')

## Word Tokenizer

In [8]:
word_tokenizer = tk.WordTokenizer()

df['poem_text'].to_csv('data.txt', sep=' ', index=False, header=False)
word_tokenizer.train('data.txt')

Training WordTokenizer ...


In [9]:
def tokenize_text_word(text):
    return word_tokenizer.tokenize(text)

df['word_t'] = df.poem_text.apply(tokenize_text_word)
df.head()

,poem_text,morph_t,word_t
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,"[عينا, ##ك, غابتا, نخيل, ساعة, ال, ##سحر, <UNK...","[عيناك, <UNK>, نخيل, ساعة, السحر, او, <UNK>, ر..."
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,"[<UNK>, لا, <UNK>, و, في, يدي, قدحي, ياليل, <U...","[انا, لا, ازال, و, في, يدي, <UNK>, <UNK>, اين,..."
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,"[علي, مقلتي, ##ك, ارتشفت, النج, ##و, ##م, وعا,...","[علي, مقلتيك, <UNK>, النجوم, <UNK>, امالي, <UN..."
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,"[اساط, ##ير, من, حشرجات, ال, ##زمان, نسيج, ال,...","[<UNK>, من, <UNK>, الزمان, <UNK>, اليد, <UNK>,..."
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,"[والت, ##ف, حول, ##ك, ساعدا, ##ي, وم, ##ال, جي...","[<UNK>, حولك, <UNK>, ومال, <UNK>, في, <UNK>, <..."


In [10]:
word_model = gensim.models.Word2Vec(
    df['word_t'],
    window=5,
    min_count=4,
    workers=8)

In [11]:
word_model.wv.most_similar(positive='ثلج')

[('عشب', 0.852849006652832),
 ('غيوم', 0.8387935757637024),
 ('رمال', 0.8382418155670166),
 ('حقول', 0.8328996896743774),
 ('فراشات', 0.829628050327301),
 ('غابات', 0.829126238822937),
 ('صحراء', 0.8254602551460266),
 ('حفيف', 0.8176856637001038),
 ('امواج', 0.8134438395500183),
 ('هدير', 0.8122962117195129)]

In [12]:
word_model.save('models/word_word2vec_model.model')

## Sentence Piece Tokenization

In [13]:
sp.SentencePieceTrainer.train(
    sentence_iterator=iter(df['poem_text'].values),
    model_prefix='models/sp_poem_model',
    vocab_size=20_000,
    pad_id=0,
    unk_id=1,
    bos_id=2,
    eos_id=3
)

sentencepiece_trainer.cc(78) LOG(INFO) Starts training with : 
trainer_spec {
  input_format: 
  model_prefix: models/sp_poem_model
  model_type: UNIGRAM
  vocab_size: 20000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_extremely_large_corpus: 0
  seed_sentencepieces_file: 
  hard_vocab_limit: 1
  use_all_vocab: 0
  unk_id: 1
  bos_id: 2
  eos_id: 3
  pad_id: 0
  unk_piece: <unk>
  bos_piece: <s>
  eos_piece: </s>
  pad_piece: <pad>
  unk_surface:  ⁇ 
  enable_differential_privacy: 0
  differential_pr

In [14]:
spp = sp.SentencePieceProcessor(model_file='models/sp_poem_model.model')

In [15]:
def tokenize_text_sp(text):
    return spp.encode(text, out_type=str)

df['sp_t'] = df.poem_text.apply(tokenize_text_sp)
df.head()

,poem_text,morph_t,word_t,sp_t
0,عيناك غابتا نخيل ساعة السحر او شرفتان راح يناي...,"[عينا, ##ك, غابتا, نخيل, ساعة, ال, ##سحر, <UNK...","[عيناك, <UNK>, نخيل, ساعة, السحر, او, <UNK>, ر...","[▁عيناك, ▁غابت, ا, ▁نخيل, ▁ساعة, ▁السحر, ▁او, ..."
1,انا لا ازال و في يدي قدحي ياليل اين تفرق الشر...,"[<UNK>, لا, <UNK>, و, في, يدي, قدحي, ياليل, <U...","[انا, لا, ازال, و, في, يدي, <UNK>, <UNK>, اين,...","[▁انا, ▁لا, ▁ازال, ▁و, ▁في, ▁يدي, ▁قد, حي, ▁يا..."
2,علي مقلتيك ارتشفت النجوم وعانقت امالي الايبة ...,"[علي, مقلتي, ##ك, ارتشفت, النج, ##و, ##م, وعا,...","[علي, مقلتيك, <UNK>, النجوم, <UNK>, امالي, <UN...","[▁علي, ▁مقلتيك, ▁ا, رتشف, ت, ▁النجوم, ▁و, عانق..."
3,اساطير من حشرجات الزمان نسيج اليد البالية رواه...,"[اساط, ##ير, من, حشرجات, ال, ##زمان, نسيج, ال,...","[<UNK>, من, <UNK>, الزمان, <UNK>, اليد, <UNK>,...","[▁اساطير, ▁من, ▁حشر, جات, ▁الزمان, ▁نسيج, ▁الي..."
4,والتف حولك ساعداي ومال جيدك في اشتهاء كالزهرة ...,"[والت, ##ف, حول, ##ك, ساعدا, ##ي, وم, ##ال, جي...","[<UNK>, حولك, <UNK>, ومال, <UNK>, في, <UNK>, <...","[▁والتف, ▁حولك, ▁ساعد, ا, ي, ▁ومال, ▁جيد, ك, ▁..."


In [16]:
sp_model = gensim.models.Word2Vec(
    df['sp_t'],
    window=5,
    min_count=4,
    workers=8)

In [17]:
sp_model.wv.most_similar(positive='ثلج')

[('جمر', 0.895128071308136),
 ('شجر', 0.8804960250854492),
 ('جرح', 0.8716637492179871),
 ('عواصف', 0.8712381720542908),
 ('نهر', 0.870846688747406),
 ('قطر', 0.8692343831062317),
 ('عسل', 0.8649084568023682),
 ('شذي', 0.8648107647895813),
 ('عصفور', 0.863004744052887),
 ('سنابل', 0.8572863936424255)]

In [18]:
sp_model.save('models/sp_word2vec_model.model')

## Save Updated Data

In [19]:
df.to_csv('all_poems_tokenized.csv', index=False)